In [41]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import training_testing_attn

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
# model = torch.compile(rnn_classes.GRUNetv2(64,64))

In [6]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [7]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ"]
raceDB = build_dataset('gru_inputs_simple_v6_test.fth', hidden_size ,state_filter=states,margin_type='sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_v1', 'last_start_price']]
(1246719, 23)
20
Latest date = 2023-05-20 00:00:00
size after state filter (131306, 25)
(131306, 25)
Latest date = 2023-05-18 00:00:00
num_features_per_dog=20


  0%|          | 0/3609 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\torch2\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/17400 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\torch2\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 17400/17400 [00:38<00:00, 451.09it/s]


number of races = 17400, number of unique dogs = 3610
0        (Auckland, 318.0)
1        (Auckland, 318.0)
2        (Auckland, 527.0)
3        (Auckland, 318.0)
4        (Auckland, 318.0)
               ...        
17395     (Waikato, 318.0)
17396     (Waikato, 318.0)
17397     (Waikato, 527.0)
17398     (Waikato, 318.0)
17399     (Waikato, 318.0)
Length: 17400, dtype: object


In [8]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 13565, Test examples 3835


In [9]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(180), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 5, 29)
start_date=datetime.date(2020, 5, 29)
period=datetime.date(2020, 11, 25)
start_date=datetime.date(2020, 11, 25)
period=datetime.date(2021, 5, 24)
start_date=datetime.date(2021, 5, 24)
period=datetime.date(2021, 11, 20)
start_date=datetime.date(2021, 11, 20)
period=datetime.date(2022, 5, 19)
start_date=datetime.date(2022, 5, 19)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 5, 29)), (datetime.date(2020, 5, 29), datetime.date(2020, 11, 25)), (datetime.date(2020, 11, 25), datetime.date(2021, 5, 24)), (datetime.date(2021, 5, 24), datetime.date(2021, 11, 20)), (datetime.date(2021, 11, 20), datetime.date(2022, 5, 19)), (datetime.date(2022, 5, 19), datetime.date(2022, 7, 31))]


In [10]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-05-29
2020-11-25
2021-05-24
2021-11-20
2022-05-19
2022-07-31
Train examples [2201, 2826, 2680, 2396, 2444, 1001]


In [11]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    # print(bi)
    # print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in tqdm(raceDB.dog_ids):
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in batch if x in dog.races.keys()]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3609 [00:00<?, ?it/s]

  0%|          | 0/3609 [00:00<?, ?it/s]

  0%|          | 0/3609 [00:00<?, ?it/s]

  0%|          | 0/3609 [00:00<?, ?it/s]

  0%|          | 0/3609 [00:00<?, ?it/s]

  0%|          | 0/3609 [00:00<?, ?it/s]

In [12]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [13]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [1557, 1594, 1489, 1492, 1400, 1111]
Train examples [1557, 1594, 1489, 1492, 1400, 1111]
Train examples [2201, 2826, 2680, 2396, 2444, 1001]
Train examples [2201, 2826, 2680, 2396, 2444, 1001]


In [14]:
raceDB.create_dogs_test_split_date()

100%|██████████| 3609/3609 [00:00<00:00, 33386.36it/s]


In [15]:
raceDB.attach_races_to_dog_inputv2()

100%|██████████| 3610/3610 [00:01<00:00, 1941.18it/s]


In [29]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = ""#[pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_x_basic = [pad_packed_sequence(pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in x]], enforce_sorted=False))for x in train_dog_input]
packed_y = ""#pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
packed_y_basic = pad_packed_sequence(pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False))

In [17]:
padded_x_basic = [pad_packed_sequence(x) for x in packed_x_basic]

In [18]:
padded_x_basic[0][0].shape

torch.Size([34, 1557, 20])

In [19]:
unpacked = unpack_sequence(packed_x_basic[0])

In [20]:
len(unpacked)

1557

In [22]:
nested = torch.nested.nested_tensor(unpacked)

C:\Users\Nick\AppData\Local\Temp\ipykernel_312\1324076212.py:1: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  nested = torch.nested.nested_tensor(unpacked)


In [21]:
encoder_layer  = nn.TransformerEncoderLayer(d_model=20,nhead=4).to('cuda:0')

In [22]:
padded_x_basic[0][0].shape

torch.Size([34, 1557, 20])

In [24]:
encoded = encoder_layer(padded_x_basic[0][0].float())

In [53]:
packed_x_basic[0][0].float().shape

torch.Size([34, 1557, 20])

In [25]:
encoded[0]

tensor([[-1.1708,  0.5108,  2.1908,  ...,  0.2228, -1.0999, -0.9056],
        [-1.4513,  0.7152,  2.1147,  ..., -0.0393, -1.0783, -0.7099],
        [-1.1686,  0.7340,  0.0943,  ...,  0.0453, -0.2615, -0.5735],
        ...,
        [ 0.9456, -0.4808,  0.0664,  ..., -0.5273, -0.5002, -0.9016],
        [ 0.8274, -0.4897, -0.1180,  ..., -0.2977, -0.3392, -1.2762],
        [-0.4448,  0.1323, -0.0369,  ..., -0.7002, -0.0393, -0.8480]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [27]:
unpad_sequence(encoded[0], padded_x_basic[0][1])

[tensor([-1.1708, -1.4513, -1.1686, -0.2120,  0.6496,  0.3790, -1.1653, -1.0864,
         -0.7685, -0.2735, -1.0294, -1.1291, -1.1102,  0.1459], device='cuda:0',
        grad_fn=<IndexBackward0>),
 tensor([0.5108, 0.7152, 0.7340, 0.7785, 0.5425, 0.5794, 0.5450, 0.3639, 0.9087,
         0.2044], device='cuda:0', grad_fn=<IndexBackward0>),
 tensor([2.1908, 2.1147, 0.0943, 1.9416, 0.9474, 1.7561, 2.1187, 2.1312, 2.1985,
         0.6009], device='cuda:0', grad_fn=<IndexBackward0>),
 tensor([-0.4003, -0.4126,  0.1401, -0.7992], device='cuda:0',
        grad_fn=<IndexBackward0>),
 tensor([0.9204, 0.1915, 0.9526, 2.2515, 2.9153, 2.0073, 1.0532, 1.3416, 1.5354,
         3.8105, 0.6398, 2.2670, 0.8582, 1.0711, 0.3113, 0.4485, 0.7538, 0.3519],
        device='cuda:0', grad_fn=<IndexBackward0>),
 tensor([-0.0934, -0.1772,  0.1373], device='cuda:0', grad_fn=<IndexBackward0>),
 tensor([-1.1421, -1.2737, -1.6256, -1.2465, -0.7016, -0.8933, -0.9594, -0.9663,
         -1.6277, -0.6409, -1.1947, -0.869

In [17]:
# train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [18]:
# [sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

In [30]:
raceDB.batches = {'num_batches':len(train_dogs),
                  'dogs':train_dogs,
                  'train_dog_input':train_dog_input,
                  'batch_races':batch_races,
                  'batch_races_ids':batch_races_ids,
                  'packed_x':packed_x,
                  'packed_x_basic':packed_x_basic,
                  'packed_y_basic':packed_y_basic, 
                  'packed_y':packed_y}

# raceDB.batches = {'num_batches':len(train_dogs),
#                     'dogs':train_dogs,
#                     'train_dog_input':train_dog_input,
#                     'batch_races':batch_races,
#                     'batch_races_ids':batch_races_ids,
#                     'packed_x':packed_x,
#                     'packed_y':packed_y}

In [48]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW attn", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric('ROI < 30', summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)
      if config.input_type == 'full':
        print('here1')
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]
      elif config.input_type == 'basic':
        print('here')
        raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
        raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
        input_size = 20

      print(f"{input_size=}")


      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.AttnNetv1(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        model = model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        optimizer.load_state_dict(model_data['optim'])
        # optimizer.to(device)
      else:
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')


      print(optimizer)

      model = model.to(device)

      print(model)

      # and use them to train the model
      try:
        training_testing_attn.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [35]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':20,
                       'stats':raceDB.stats_cols,
                       'races':states,
                       'latest_date':raceDB.latest_date,
                       'input_type':'basic',
                       'num_layers':1,
                       'batch_size': 750,
                       'dropout': 0.3,
                       'epochs': 10_000,
                       'learning_rate': 0.0004,
                       'optimizer': 'adamW',
                       'f0_layer_size':128,
                       'f1_layer_size':64}

In [54]:
importlib.reload(training_testing_attn)
importlib.reload(rnn_classes)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [34]:
torch.cuda.empty_cache()

In [24]:
for race in raceDB.racesDict.values():
    for dog in race.dogs:
        try:
            del dog.stats
        except:
            pass

torch.cuda.empty_cache()

In [36]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [37]:
torch.cuda.empty_cache()

In [55]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 20, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_v1', 'last_start_price']]", 'races': ['NZ'], 'latest_date': '2023-05-18T00:00:00', 'input_type': 'basic', 'num_layers': 1, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0004, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64}
10000
{'hidden_size': 20, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_v1', 'last_start_price']]", 'races': ['NZ'], 'latest_date': '2023-05-18T00:00:00', 'input_type': 'basic', 'num_layers': 1, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0004, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_s

  0%|          | 0/10000 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f0_layer_size": {"values": [1024]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['full','basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    # "hidden_size": {"value":hidden_size}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [1500]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        # "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00008,
            "max": 0.0015,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [7]
        },
        "batch_before_backwards": {
            'values': [7]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)